In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome,
                           perform_experiments, ema_logging,Policy, MultiprocessingEvaluator)

from ema_workbench import Samplers

from dike_model_function import DikeNetwork  # @UnresolvedImport

from problem_formulation import get_model_for_problem_formulation

from ema_workbench.em_framework.salib_samplers import get_SALib_problem

from SALib.analyze import sobol

import pickle

In [44]:
ema_logging.log_to_stderr(ema_logging.INFO);

In [39]:
dike_model, planning_steps = get_model_for_problem_formulation(2)

In [41]:
planning_steps

array([0, 1, 2])

In [51]:
uncertainties = dike_model.uncertainties
outcomes = dike_model.outcomes
levers = dike_model.levers

problem = get_SALib_problem(uncertainties)
print(problem)

{'num_vars': 19, 'names': ['A.0_ID flood wave shape', 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail', 'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate', 'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax', 'A.5_Brate', 'A.5_pfail', 'discount rate 0', 'discount rate 1', 'discount rate 2'], 'bounds': [(0, 133), (30, 350), (0, 3), (0, 1), (30, 350), (0, 3), (0, 1), (30, 350), (0, 3), (0, 1), (30, 350), (0, 3), (0, 1), (30, 350), (0, 3), (0, 1), (0, 4), (0, 4), (0, 4)]}


In [64]:
#for unc in dike_model.uncertainties:
#    print(repr(unc))

In [65]:
#for out in dike_model.outcomes:
#    print(repr(out))

In [60]:
for lev in dike_model.levers:
    print(repr(lev.name))

'0_RfR 0'
'0_RfR 1'
'0_RfR 2'
'1_RfR 0'
'1_RfR 1'
'1_RfR 2'
'2_RfR 0'
'2_RfR 1'
'2_RfR 2'
'3_RfR 0'
'3_RfR 1'
'3_RfR 2'
'4_RfR 0'
'4_RfR 1'
'4_RfR 2'
'EWS_DaysToThreat'
'A.1_DikeIncrease 0'
'A.1_DikeIncrease 1'
'A.1_DikeIncrease 2'
'A.2_DikeIncrease 0'
'A.2_DikeIncrease 1'
'A.2_DikeIncrease 2'
'A.3_DikeIncrease 0'
'A.3_DikeIncrease 1'
'A.3_DikeIncrease 2'
'A.4_DikeIncrease 0'
'A.4_DikeIncrease 1'
'A.4_DikeIncrease 2'
'A.5_DikeIncrease 0'
'A.5_DikeIncrease 1'
'A.5_DikeIncrease 2'


In [66]:
### True, use results in pickle file; False, run MultiprocessingEvaluator
use_pickle1 = True

if use_pickle1:
    with open('data/formulation_results.pickle', 'rb') as filehandler:
        results = pickle.load(filehandler)

else:
    # pass the policies list to EMA workbench experiment runs
    n_scenarios = 10
    with MultiprocessingEvaluator(dike_model) as evaluator:
        results = evaluator.perform_experiments(n_scenarios, uncertainty_sampling=Samplers.SOBOL)
        
    experiments, outcomes = results

    # Save results in Pickle file
    with open('data/formulation_results.pickle', 'wb') as filehandler:
        pickle.dump(results, filehandler)

NameError: name 'pickle' is not defined

In [50]:
n_exp = 10

experiments_sobol, outcomes_sobol = perform_experiments(dike_model, scenarios=n_exp,
                                                        uncertainty_sampling=Samplers.SOBOL)

/Users/marcpeterstreng/opt/miniconda3/envs/py39/lib/python3.9/site-packages/SALib/sample/saltelli.py:94: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (1000) is equal to `2^n`.
        
  warnings.warn(msg)
[MainProcess/INFO] performing 40000 scenarios * 1 policies * 1 model(s) = 40000 experiments
  0%|                                                | 0/40000 [00:00<?, ?it/s][MainProcess/INFO] performing experiments sequentially
[MainProcess/ERROR] 'DikeIncrease 0'
Traceback (most recent call last):
  File "/Users/marcpeterstreng/opt/miniconda3/envs/py39/lib/python3.9/site-packages/ema_workbench/em_framework/experiment_runner.py", line 92, in run_experiment
    model.run_model(scenario, policy)
  File "/Users/marcpeterstreng/opt/miniconda3/envs/py39/lib/python3.9/site-packages/ema_workbench/util/ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "/Users/marcpeterstreng/opt/miniconda3/envs/py39/lib/python3.9/si

EMAError: exception in run_model
Caused by: KeyError: 'DikeIncrease 0'

In [48]:
for outcome in outcomes:
    print(repr(unc)) 

CategoricalParameter('A.5_Brate', [0, 1, 2])
CategoricalParameter('A.5_Brate', [0, 1, 2])
CategoricalParameter('A.5_Brate', [0, 1, 2])
CategoricalParameter('A.5_Brate', [0, 1, 2])
CategoricalParameter('A.5_Brate', [0, 1, 2])


In [ ]:
prey_final_sobol = outcomes_sobol['prey'][:,0,-1]
prey_mean_sobol = np.mean(outcomes_sobol['prey'][:,0,:],axis=1)
prey_std_sobol = np.std(outcomes_sobol['prey'][:,0,:],axis=1)

Si = sobol.analyze(problem, prey_mean_sobol, calc_second_order=True, print_to_console=True)

In [ ]:
scen1 = {
'discount rate 0'
'discount rate 1'
'discount rate 2'
'A.0_ID flood wave shape'
'A.1_Bmax'
'A.1_pfail'
'A.1_Brate'
'A.2_Bmax'
'A.2_pfail'
'A.2_Brate'
'A.3_Bmax'
'A.3_pfail'
'A.3_Brate'
'A.4_Bmax'
'A.4_pfail'
'A.4_Brate'
'A.5_Bmax'
'A.5_pfail'
'A.5_Brate'}


In [ ]:
    # no dike increase, no warning, none of the rfr
    zero_policy = {'DaysToThreat': 0}
    zero_policy.update({'DikeIncrease {}'.format(n): 0 for n in planning_steps})
    zero_policy.update({'RfR {}'.format(n): 0 for n in planning_steps})
    pol0 = {}

    for key in dike_model.levers:
        s1, s2 = key.name.split('_')
        pol0.update({key.name: zero_policy[s2]})

    policy0 = Policy('Policy 0', **pol0)